In [1]:
import numpy as np
import pandas as pd
import time

import serial
import serial.tools.list_ports

from pathlib import Path

# import workbench.config.config'
from workbench.config.config import initialize
from workbench.utils.utils import create_filepaths

# Setting up file paths

In [4]:
models_dir = initialize()

In [5]:
model_name = "shufflenetv2tiny_0.05_96_c3_o2_f24l1024"

In [6]:
models_path, models_summary_path, models_image_path, models_layer_df_path, models_tf_path, models_tflite_path, models_tflite_opt_path = create_filepaths(model_name)

i:\tinyml\tiny_cnn\models


In [7]:
# create path to cpp files
arduino_csv_path = models_path.joinpath("arduino_inference.csv")
model_cpp_path = models_path.joinpath("model.cpp")
model_header_path = models_path.joinpath("model.h")



In [8]:
# get the list of recently created models
model_names = []
with open("deployment_list.txt", "r") as file:
    lines = file.readlines()
    for line in lines:
        if len(line) < 2:
            pass
        else:
            model_names.append(line.strip())

model_names

['mobilenetv1_0.1_96_c3_o2_l2.MV1']

In [9]:
model_name = 'mobilenetv1_0.1_96_c3_o2_l2.MV1'

In [10]:
models_h_path = models_dir.joinpath(model_name, "model.h")
models_c_path = models_dir.joinpath(model_name, "model.cpp")

# Preparation for compilation on Arduino

In [11]:
# copy model files to cpp project
# TODO

# Compile the project and upload it to Arduino

In [12]:
!arduino-cli compile --fqbn arduino:mbed_nano:nano33ble deployment/model_test -v

Der Befehl "arduino-cli" ist entweder falsch geschrieben oder
konnte nicht gefunden werden.


In [ ]:
!arduino-cli upload -p COM4 --fqbn arduino:mbed_nano:nano33ble deployment/model_test

# Working with Arduino

In [59]:
ports = serial.tools.list_ports.comports()
ports

In [60]:
[port.manufacturer for port in ports]

['Microsoft']

In [61]:
port = ports[0].device

In [62]:
# Open port
arduino = serial.Serial(port, baudrate=115200, timeout=1)

In [63]:
arduino.close()

In [64]:
def handshake_arduino(
    arduino, sleep_time=1, print_handshake_message=False, handshake_code=000
):
    """Make sure connection is established by sending
    and receiving bytes."""
    # Close and reopen
    arduino.close()
    arduino.open()

    # Chill out while everything gets set
    time.sleep(sleep_time)

    # Set a long timeout to complete handshake
    timeout = arduino.timeout
    arduino.timeout = 2

    # Read and discard everything that may be in the input buffer
    _ = arduino.read_all()

    # Send request to Arduino
    arduino.write(bytes([handshake_code]))

    # Read in what Arduino sent
    handshake_message = arduino.read_until()

    # Send and receive request again
    arduino.write(bytes([handshake_code]))
    handshake_message = arduino.read_until()

    # Print the handshake message, if desired
    if print_handshake_message:
        print("Handshake message: " + handshake_message.decode())

    # Reset the timeout
    arduino.timeout = timeout


In [65]:
# Call the handshake function
handshake_arduino(arduino, print_handshake_message=True)

Handshake message: 


In [66]:
def parse_raw(raw):
    """Parse bytes output from Arduino."""
    raw = raw.decode()
    if raw[-1] != "\n":
        raise ValueError(
            "Input must end with newline, otherwise message is incomplete."
        )

    line = raw.rstrip()

    return line

It is important to always close the ports when they are not used!

In [67]:
def get_arduino_lines(n_data = 1000, port="COM3"):
    arduino = serial.Serial(port, baudrate=115200, timeout=1)

    handshake_arduino(arduino, print_handshake_message=True)
    
    # arduino.close()
    # arduino.open()

    lines =[]
    i = 0

    while i < n_data:
        try:
            raw = arduino.read_until()
            raw = parse_raw(raw)
            print(raw)
            lines.append(raw)
            i +=1
        except:
            "Print"
    arduino.close()

    return lines

In [68]:
arduino.close()

In [69]:
scan_lines = get_arduino_lines(port=port)

Handshake message: 


# Cleanup the received data

In [ ]:
def create_MCU_inference_df(lines):
    clean_lines = []
    for line in lines:
        clean_lines.append(str(line).replace("lu", ""))#.replace(").", ""))

    #clean_lines
    lines_df = pd.DataFrame(clean_lines)

    split_df = lines_df[0].str.split(",",expand=True)
    split_df.dropna(inplace=True)
    # setting first row as headers
    split_df.columns = split_df.iloc[0]
    split_df = split_df[1:]
    split_df.columns = [c.strip('"') for c in split_df.columns.values.tolist()]

    # data cleaning
    split_df = split_df[split_df['Ticks'] != '"Ticks"']
    split_df = split_df.fillna(0)
    split_df["Ticks"] = split_df["Ticks"].astype(int)
    split_df["ms"] = split_df["Ticks"]/ 1000.0

    # finding start and end of each inference
    # each inference starts and ends with quantization of the input or output tensor
    split_df["next_Tag"] = split_df["Tag"].shift()
    split_df["prev_Tag"] = split_df["Tag"].shift(-1)
    split_df["start"] = (split_df["next_Tag"] == split_df["Tag"])
    split_df["end"] = (split_df["prev_Tag"] == split_df["Tag"])

    # create markers for the different inference cycles
    inference = []
    i =1 
    for row in split_df['start']:
        if row == True:
            inference.append(f"inference_{i}")
            i = i+1
        else:
            inference.append(None)

    split_df['inference'] = inference
    split_df['inference'].fillna(method="ffill", inplace=True)
    
    split_df.drop(["start", "end","next_Tag", "prev_Tag"], axis=1, inplace=True)

    return split_df

    


In [ ]:
inference_df = create_MCU_inference_df(scan_lines)
inference_df

,Event,Tag,Ticks,ms,inference
2,0,RESHAPE,23,0.023,None
3,1,TRANSPOSE,719,0.719,None
4,2,RESHAPE,23,0.023,None
5,3,SPLIT,155,0.155,None
6,4,CONV_2D,1686,1.686,None
...,...,...,...,...,...
995,19,SPLIT,166,0.166,inference_6
996,20,CONV_2D,1674,1.674,inference_6
997,21,DEPTHWISE_CONV_2D,821,0.821,inference_6
998,22,CONV_2D,1685,1.685,inference_6


In [ ]:
inference_1_df = inference_df[inference_df["inference"]== "inference_1"]
inference_1_df.reset_index(inplace=True,drop=True)
inference_1_df.reset_index(inplace=True)
inference_1_df

,index,Event,Tag,Ticks,ms,inference
0,0,32,QUANTIZE,5750,5.750,inference_1
1,1,33,CONV_2D,70147,70.147,inference_1
2,2,34,MAX_POOL_2D,6495,6.495,inference_1
3,3,35,CONV_2D,8309,8.309,inference_1
4,4,36,DEPTHWISE_CONV_2D,1399,1.399,inference_1
...,...,...,...,...,...,...
166,166,27,CONV_2D,61690,61.690,inference_1
167,167,28,MEAN,48517,48.517,inference_1
168,168,29,FULLY_CONNECTED,277,0.277,inference_1
169,169,30,SOFTMAX,55,0.055,inference_1


In [ ]:
inference_1_time = inference_1_df["ms"].sum()
inference_1_time

489.67500000000007

In [ ]:
inference_2_df = inference_df[inference_df["inference"]== "inference_2"]
inference_2_df

,Event,Tag,Ticks,ms,inference
243,32,QUANTIZE,5750,5.750,inference_2
244,33,CONV_2D,70147,70.147,inference_2
245,34,MAX_POOL_2D,6495,6.495,inference_2
246,35,CONV_2D,8309,8.309,inference_2
247,36,DEPTHWISE_CONV_2D,1399,1.399,inference_2
...,...,...,...,...,...
377,166,CONV_2D,61697,61.697,inference_2
378,167,MEAN,48500,48.500,inference_2
379,168,FULLY_CONNECTED,287,0.287,inference_2
380,169,SOFTMAX,55,0.055,inference_2


In [ ]:
inference_2_time = inference_2_df["ms"].sum()
inference_2_time

362.855